[View in Colaboratory](https://colab.research.google.com/github/zhh210/flea_market/blob/master/Tree_visualization.ipynb)

In [1]:
!pip3 install spacy

!pip3 install toyplot
!python3 -m spacy download en_core_web_lg
!pip3 install ete3

!git clone https://github.com/eaton-lab/toytree.git
!mv toytree tmp
!pip3 install tmp/

# !apt get-install python-pydot python-pydot-ng graphviz
# !pip3 install graphviz# !pip3 install pygraphviz

    100% |████████████████████████████████| 22.0MB 1.6MB/s 
    100% |████████████████████████████████| 122kB 21.5MB/s 
    100% |████████████████████████████████| 1.2MB 6.8MB/s 
    100% |████████████████████████████████| 194kB 23.6MB/s 
    100% |████████████████████████████████| 153kB 22.3MB/s 
    100% |████████████████████████████████| 604kB 13.7MB/s 
    100% |████████████████████████████████| 317kB 16.9MB/s 
    100% |████████████████████████████████| 450kB 19.0MB/s 
    100% |████████████████████████████████| 51kB 17.0MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/60/0b/bb/7

In [2]:

import toytree
import toyplot

newick = """(41954_cyathophylloides:0.00008888031167776559,(((32082_przewalskii:
0.00259326350604092027,33588_przewalskii:0.00247133969857381459)100:
0.03587422609749137820,(33413_thamno:0.00565358258838428562,(30556_thamno:
0.00653218253974622003,((40578_rex:0.00335406134690998791,35855_rex:
0.00339963302433546593)100:0.00222999650239191338,(35236_rex:0.00580524693403740473,
(39618_rex:0.00096208118162745867,38362_rex:0.00109218434613194098)100:
0.00617527349892385037)96:0.00073890038051916795)99:0.00078336549990502716)100:
0.00103379657491441167)100:0.00538723112354794632)100:0.00297626149201316807,
(29154_superba:0.00634236826447889986,30686_cyathophylla:0.00669944988923529706)
100:0.00237994755604001816)100:0.00941020878048287081,41478_cyathophylloides:
0.00005282184335845886);"""
newick = "((a:1,b:1)90:2,(c:3,(d:1, e:1)100:2)100:1)100;"
## load newick string to create a Toytree class object
tre3 = toytree.tree(newick)
canvas, axes = tre3.draw(width=400, node_labels=True,mmarker=toyplot.marker.create(shape="r3x1", angle=0, size=15, label="1.2", mstyle={"fill":"white"}))


e d c b a idx: 8
name: i8
dist: 0
support: 100
height: 4 idx: 7
name: i7
dist: 2
support: 90
height: 2 idx: 6
name: i6
dist: 1
support: 100
height: 3 idx: 5
name: i5
dist: 2
support: 100
height: 1 idx: 4
name: a
dist: 1
support: 100
height: 1 idx: 3
name: b
dist: 1
support: 100
height: 1 idx: 2
name: c
dist: 3
support: 100
height: 0 idx: 1
name: d
dist: 1
support: 100
height: 0 idx: 0
name: e
dist: 1
support: 100
height: 0

In [3]:
# !pip3 install docs
import numpy as np
# import docs
np.random.seed(1234)
# edges = docs.prufer_tree(numpy.random.choice(4, 12))
edges = tre3.edges
print(edges)
layout = toyplot.layout.Buchheim()
toyplot.graph(edges,vsize=30, layout=layout, width=800, height=400,mmarker=[toyplot.marker.create(shape="r3x1", angle=0, size=15, label='advmod', mstyle={"fill":"white"})]*8)

[[7 4]
 [7 3]
 [8 7]
 [6 2]
 [5 1]
 [5 0]
 [6 5]
 [8 6]]


(<toyplot.canvas.Canvas at 0x7ff2e7264f60>,
 <toyplot.mark.Graph at 0x7ff2e6ffb828>)

advmod advmod advmod advmod advmod advmod advmod advmod 0 1 2 3 4 5 6 7 8

In [4]:
import spacy
nlp = spacy.load('en_core_web_lg')
sent = nlp(u"IBM has very strong performance in Q2 but is overshadowed by its subsidiaries which are not doing well.")

def visualize(doc):
    edge, types = [], []
    Q = [next(doc.sents).root]
    colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))
    while Q:
      node = Q.pop(0)
      for j in node.children:
        edge.append([node,j])
        types.append(j.dep_)
        Q.append(j)

    make_marker = lambda x: toyplot.marker.create(shape="r3x1", angle=0, size=10, label=x, mstyle={"fill":"white"})

    toyplot.graph(np.array(edge),vsize=30,layout=layout,mmarker=[make_marker(x) for x in types])

visualize(sent)

nsubj dobj cc conj punct amod prep auxpass agent advmod pobj pobj poss relcl nsubj aux neg advmod IBM has very strong performance in Q2 but is overshadowed by its subsidiaries which are not doing well .

In [19]:
import spacy
from spacy.tokens import Doc, Span, Token

class SentimentTree(object):
	'''Functions related to SentimentTree'''
	@staticmethod
	def sentiment(sent,root=None):
		Token.set_extension('tree_score',default=0,force=True)
		Token.set_extension('_multiplier',default=1,force=True)
		if root is None: root = sent.root
		Q = []
		C = [root]
		while C:
			node = C.pop(0)
			node._.tree_score = node._.valence
			Q.append(node)
			for j in node.children: C.append(j)

		for j in reversed(Q[1:]):
			if j.dep_ in ('neg'): 
				j.head._.tree_score *= N_SCALAR
			else:
				j.head._.tree_score += j._.tree_score*j.head._._multiplier
				if j._.boost_val: j.head._.tree_score += j._.boost_val*j.head._._multiplier

			if j.dep_ in ('cc') and j.text.lower() in ('but','however'):
				j.head._.tree_score *= 1.5
				j.head._._multiplier = 0.5

		return root._.tree_score

def plot_tree(doc,mode='post',wt_score=True):
	'''Visualize parsing tree.'''
	attr = 'tree_score' if mode == 'post' else 'valence'
	import toytree, toyplot
	import numpy as np
	layout = toyplot.layout.Buchheim()
	for sent in doc.sents:
	    edges, types = [], []
	    Q = [sent.root]
	    
	    while Q:
	      	node = Q.pop(0)
	      	for j in node.children:
	      	    n1, n2 = node, j
	      	    # n1 += '\n' + str(node._.get(attr)) if wt_score and node._.get(attr) != 0 else ''
	      	    # n2 += '\n' + str(j._.get(attr)) if wt_score and j._.get(attr) != 0 else ''
	      	    edges.append([n1, n2])
	      	    types.append(j.dep_)
	      	    Q.append(j)


	    make_marker = lambda x: toyplot.marker.create(shape="r3x1", angle=0, size=10, label=x, mstyle={"fill":"white"})
	    from itertools import cycle
	    # colors = [i for i,j in zip(cycle(['mediumseagreen','blue','red']),range(len(sent)))]
	    colors = [toyplot.color.rgba(1, 0, 0, abs(i._.get(attr))/4) if i._.get(attr) < 0 else (toyplot.color.rgba(0.4, 0.76078431, 0.64705882, abs(i._.get(attr))/4) if i._.get(attr) > 0 else toyplot.color.rgba(0.70196078, 0.70196078, 0.70196078, 0.2)) for i in sent]
	    yield toyplot.graph(np.array(edges),vcolor=colors,vsize=30,layout=layout,mmarker=[make_marker(x) for x in types])

      
def prepare(doc,lvalence,lboost):
    'Assign necessary scores to doc'
    Token.set_extension('valence',default=0,force=True)
    Token.set_extension('boost_val',default=None,force=True)
    for i,j,k in zip(doc,lvalence,lboost):
        i._.valence = j
        i._.boost_val = k
    
  
  
      
def test_plot():
    print('Testing functionality of generating tree visualization.')
    nlp = spacy.load('en_core_web_lg')
    doc = nlp("The company also faces liquidity pressures due to its large negative free cash flow and the pending maturities of convertible bonds ($230 million in November 2018 and $920 million in March 2019).")
    for ent in doc.ents: ent.merge()
    prepare(doc,[0,0,0,0,0,-1.3,0,0,0,0,-2.7,2.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[None]*len(doc))
    print('SentimentTree score: ',SentimentTree.sentiment(next(doc.sents)))
    trees = plot_tree(doc)
    canvas, axes, mark = next(trees)
    import toyplot.pdf
    # import pdb; pdb.set_trace()
    toyplot.pdf.render(canvas,'test.pdf')
    from google.colab import files
    files.download('test.pdf')
    return canvas
    
    
test_plot()

Testing functionality of generating tree visualization.
SentimentTree score:  -1.7000000000000004


Exception: ignored

In [0]:
!git clone https://github.com/openai/generating-reviews-discovering-sentiment.git
import os
os.chdir('generating-reviews-discovering-sentiment/')

Cloning into 'generating-reviews-discovering-sentiment'...
remote: Counting objects: 59, done.
remote: Total 59 (delta 0), reused 0 (delta 0), pack-reused 59
Unpacking objects: 100% (59/59), done.


In [0]:
import html
def preprocess(text, front_pad='\n ', end_pad=' '):
    text = html.unescape(text)
    text = text.replace('\n', ' ').strip()
    text = front_pad+text+end_pad
    text = text.encode()
    return text
 
preprocess('demo!')


b'\n demo! '

In [0]:
import os

# from encoder import Model

# model = Model()
text = 'The overall performance is great but its subsidiary is not doing well.'.split()
# text = ['the','market','is', 'good','but','IBM','is','suffering']
text_features = model.transform(text)
text_features[:,2388]

9.997 seconds to transform 12 examples


array([ 0.08647688, -0.14761488,  0.06720525,  0.05931341,  0.554182  ,
       -0.12429999,  0.06046563,  0.07367658,  0.05931341, -0.00414762,
        0.06756548,  0.01692243], dtype=float32)

In [0]:
exec(open('sst_binary_demo.py','r').read())